### Data

In [ ]:
!pip3 install pycocotools

In [ ]:
# Importing required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import data_utils
import img_utils
from data_utils import get_transform, BloodCellDataset
from bcnet import get_bcnet
from train_eval_utils import (
    hocv_model, 
    kfcv_model, 
    plot_metrics_per_epoch, 
    train_model, 
    eval_model, 
    save_model,
    load_model,
    model_predict
)

In [ ]:
#!unzip data.zip -d /storage

In [ ]:
# Plot one sample image

full_dataset = data_utils.BloodCellDataset('data')
image, target = full_dataset[0]
img_utils.plot_img_w_box(image, target)

Reading in 'annotations.csv' and checking for duplicates, followed by dropping them:

In [ ]:
anno_df = pd.read_csv('data/annotations.csv')
dup_df = data_utils.get_duplicates(anno_df)
to_drop=[dup_df.loc[(dup_df['image']=='image-1.png') & (dup_df['label']=='rbc'),:].index[0]]
to_drop.append(dup_df.loc[(dup_df['image']=='image-100.png') & (dup_df['label']=='wbc'),:].index[0])
to_drop.append(dup_df.loc[(dup_df['image']=='image-104.png') & (dup_df['label']=='wbc'),:].index[0])
to_drop.append(dup_df.loc[(dup_df['image']=='image-114.png') & (dup_df['label']=='wbc'),:].index[0])
clean_anno_df = anno_df.drop(to_drop)
clean_anno_df['label'] = np.where(clean_anno_df['label']=='rbc', 1, 2)
#clean_anno_df.to_csv('data/clean_anno.csv', index=False)

Visualizing the distribution of RBCs to WBCs across all the images

In [ ]:
data_utils.get_class_distribution('data/clean_anno.csv', ['image','label'])

### Cross Validation

In [ ]:
# 80-20 train-test split
train_indices, test_indices = train_test_split(range(100), test_size=0.2, random_state=42)

In [ ]:
# Hold-out cross validation (uncomment and run only if you wish to do this, if not use kfold)

# frcnn = get_bcnet('retina', 3)
# hocv_model(
#     model=frcnn, 
#     dir_name='/storage/data', 
#     train_idxs=train_indices, 
#     val_idxs=test_indices, 
#     train_transforms=get_transform(True), 
#     val_transforms=get_transform(False), 
#     num_epochs=20, 
#     batch_size=2, 
#     learning_rate=0.01
#     )

In [ ]:
# Hyperparameter tuning here

model_to_tune = 'frcnn'  # Adjust to which model you wish to tune for

train_hist, val_hist = kfcv_model(
    model_type=model_to_tune,
    dir_name='/storage/data',
    train_idxs=train_indices,
    train_transforms=get_transform(True),
    val_transforms=get_transform(False),
    num_epochs=30,
    batch_size=2,
    learning_rate=0.1,
    k_folds=5
)

plot_metrics_per_epoch(train_hist, val_hist)

From our the validation scores of each model, we will use these hyperparameters for the final training of the three models:

* Faster R-CNN
    * Number of epochs: 15
    * Initial learning rate: 0.1
    * Batch size: 2

* RetinaNet
    * Number of epochs: 15
    * Initial learning rate: 0.05
    * Batch size: 2

* SSD
    * Number of epochs: 15
    * Initial learning rate: 0.005
    * Batch size: 2
    
   

### Training and evaluation

In [ ]:
# Train all three models

frcnn = get_bcnet('frcnn')
retinanet = get_bcnet('retina')
ssd_net = get_bcnet('ssd')

frcnn = train_model(
    model=frcnn,
    dir_name='/storage/data',
    train_idxs=train_indices,
    transforms=get_transform(train=True),
    num_epochs=15,
    batch_size=2,
    learning_rate=0.1
)

retinanet = train_model(
    model=retinanet,
    dir_name='/storage/data',
    train_idxs=train_indices,
    transforms=get_transform(train=True),
    num_epochs=15,
    batch_size=2,
    learning_rate=0.05
)

ssd_net = train_model(
    model=ssd_net,
    dir_name='/storage/data',
    train_idxs=train_indices,
    transforms=get_transform(train=True),
    num_epochs=15,
    batch_size=2,
    learning_rate=0.005
)

In [ ]:
# Evaluate models on test set

frcnn_test = eval_model(
    model=frcnn,
    dir_name='/storage/data',
    test_idxs=test_indices,
    test_transforms=get_transform(train=False)
)

retina_test = eval_model(
    model=retinanet,
    dir_name='/storage/data',
    test_idxs=test_indices,
    test_transforms=get_transform(train=False)
)

ssd_test = eval_model(
    model=ssd_net,
    dir_name='/storage/data',
    test_idxs=test_indices,
    test_transforms=get_transform(train=False)
)

for res in zip(['Faster R-CNN', 'RetinaNet', 'SSD'], [frcnn_test, retina_test, ssd_test]):
    print(
        '{0} --> COCO-mAP: {1:.4f}, mAP-50: {2:.4f}, mAP-75: {3:.4f}'.format(
            res[0], res[1]['coco_map'], res[1]['map_50'], res[1]['map_75']
            )
        )

### Generate predictions from final models

Take note that we use non-maximum suppression here to remove "excess" bounding boxes. 

In [ ]:
test_dataset = BloodCellDataset('/storage/data', get_transform(train=False))
image, actual = test_dataset[0]
predicted = model_predict(frcnn_test, image)
img_utils.plot_img_w_box(image, predicted, nms=True, iou_threshold=0.4)

In [ ]:
img_utils.plot_img_w_box(image, actual)

In [ ]:
# Saving models

save_model(model=frcnn, save_path="saved_models/best_frcnn.pth")
save_model(model=retinanet, save_path="saved_models/best_retina.pth")
save_model(model=ssd_net, save_path="saved_models/best_ssd.pth")

In [ ]:
# Example on how to load a model from saved path
new_frcnn = get_bcnet('frcnn', 3, 1)
load_model(new_frcnn, "saved_models/best_frcnn.pth")